# Classic NN

In [2]:
#scikit learn ensembe workflow for binary probability
import time; start_time = time.time()
import numpy as np
import pandas as pd
from sklearn import ensemble
import xgboost as xgb
from sklearn.metrics import log_loss, make_scorer
import random; random.seed(7)
from math import*
from decimal import Decimal
import os
from scipy import fftpack
import csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

/root/anaconda2/lib/python2.7/site-packages/numpy/core/machar.py:127: RuntimeWarning: overflow encountered in add
  a = a + a
/root/anaconda2/lib/python2.7/site-packages/numpy/core/machar.py:129: RuntimeWarning: invalid value encountered in subtract
  temp1 = temp - a
/root/anaconda2/lib/python2.7/site-packages/numpy/core/machar.py:138: RuntimeWarning: invalid value encountered in subtract
  itemp = int_conv(temp-a)
/root/anaconda2/lib/python2.7/site-packages/numpy/core/machar.py:162: RuntimeWarning: overflow encountered in add
  a = a + a
/root/anaconda2/lib/python2.7/site-packages/numpy/core/machar.py:164: RuntimeWarning: invalid value encountered in subtract
  temp1 = temp - a
/root/anaconda2/lib/python2.7/site-packages/numpy/core/machar.py:171: RuntimeWarning: invalid value encountered in subtract
  if any(temp-a != zero):
Using TensorFlow backend.
/root/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0

## Metrics definition

In [2]:
def histogram_intersection(h1, h2, bins):
   bins = np.diff(bins)
   sm = 0
   for i in range(len(bins)):
       sm += min(bins[i]*h1[i], bins[i]*h2[i])
   return sm

def manhattan_distance(x,y):
    return sum(abs(a-b) for a,b in zip(x,y))
 
def nth_root(value, n_root):
    root_value = 1/float(n_root)
    return round (Decimal(value) ** Decimal(root_value),3)
 
def minkowski_distance(x,y,p_value):
     return nth_root(sum(pow(abs(a-b),p_value) for a,b in zip(x, y)),p_value)
    
def square_rooted(x):
    return round(sqrt(sum([a*a for a in x])),3)
 
def cosine_similarity(x,y):
    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return round(numerator/float(denominator),3)    

def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

def fft(img):
    # Take the fourier transform of the image.
    F1 = fftpack.fft2(img)
 
    # Now shift the quadrants around so that low spatial frequencies are in
    # the center of the 2D fourier transformed image.
    F2 = fftpack.fftshift( F1 )
 
    # Calculate a 2D power spectrum
    psd2D = np.abs( F2 )**2
    return psd2D

## Load original datasets

In [3]:
#Load data
train = pd.read_json("./train.json")
test = pd.read_json("./test.json")

# train.head(3)

## Compute train and test metrics

In [4]:
# Train data
metrics = []

if os.path.isfile('./train_metrics.csv'):
    print 'Already done...'
else:
    for idx in range(0,train.shape[0]):
        row = []
        img1 = train.loc[idx, ['band_1', 'band_2']]
        target = (train.loc[idx, ['is_iceberg']])[0]
        row.append(target)
        
        id = (train.loc[idx, ['id']])[0]
        row.append(id)

        img1 = np.stack([img1['band_1'], img1['band_2']], -1).reshape(75, 75, 2)
        band1 = img1[:, :, 0]
        band1 = (band1 + abs(band1.min())) / np.max((band1 + abs(band1.min())))

        band2 = img1[:, :, 1]
        band2 = (band2 + abs(band2.min())) / np.max(band2 + abs(band2.min()))

        fft_band1 = fft(band1)
        fft_band2 = fft(band2)

        hband1, hbins = np.histogram(band1, bins=256, normed=True)
        hband2, hbins = np.histogram(band2, bins=256, normed=True)

        fft_hband1, hbins = np.histogram(fft_band1, bins=256, normed=True)
        fft_hband2, hbins = np.histogram(fft_band2, bins=256, normed=True)

        hist_band1 = np.array([hband1]).ravel()
        series_band1 = pd.Series(hist_band1)
        hist_band2 = np.array([hband2]).ravel()
        series_band2 = pd.Series(hist_band2)

        fft_hist_band1 = np.array([fft_hband1]).ravel()
        fft_series_band1 = pd.Series(fft_hist_band1)
        fft_hist_band2 = np.array([fft_hband2]).ravel()
        fft_series_band2 = pd.Series(fft_hist_band2)

        diff = series_band1 - series_band2
        distance = np.sqrt(np.dot(diff, diff))
        row.append(distance)

        inter = histogram_intersection(hband1, hband2, hbins)
        row.append(inter)

        diff = fft_series_band1 - fft_series_band2
        distance = np.sqrt(np.dot(diff, diff))
        row.append(distance)

        inter = histogram_intersection(fft_hband1, fft_hband2, hbins)
        row.append(inter)

        manhattan = manhattan_distance(np.array([band1]).ravel(),np.array([band2]).ravel())
        row.append(manhattan)

        minkowski = minkowski_distance(np.array([band1]).ravel(),np.array([band2]).ravel(), 3)
        row.append(minkowski)

        cos_simil = cosine_similarity(np.array([band1]).ravel(),np.array([band2]).ravel())
        row.append(cos_simil)

        jaccard_simil = jaccard_similarity(np.array([band1]).ravel(),np.array([band2]).ravel())
        row.append(jaccard_simil)
        
        metrics.append(row)

    header = 'target', 'id', 'normal_dist','normal_intersec', 'fft_dist','fft_intersec', 'manhattan_dist', 'minkowski_dist', 'cosine_simil', 'jaccard_simil'
            
    with open("./train_metrics.csv", "wb") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(metrics)

Already done...


In [5]:
# Test data
metrics = []

if os.path.isfile('./test_metrics.csv'):
    print 'Already done...'
else:
    for idx in range(0,train.shape[0]):
        row = []
        img1 = train.loc[idx, ['band_1', 'band_2']]
        
        row.append(0)
        
        id = (train.loc[idx, ['id']])[0]
        row.append(id)
        
        img1 = np.stack([img1['band_1'], img1['band_2']], -1).reshape(75, 75, 2)
        band1 = img1[:, :, 0]
        band1 = (band1 + abs(band1.min())) / np.max((band1 + abs(band1.min())))

        band2 = img1[:, :, 1]
        band2 = (band2 + abs(band2.min())) / np.max(band2 + abs(band2.min()))

        fft_band1 = fft(band1)
        fft_band2 = fft(band2)

        hband1, hbins = np.histogram(band1, bins=256, normed=True)
        hband2, hbins = np.histogram(band2, bins=256, normed=True)

        fft_hband1, hbins = np.histogram(fft_band1, bins=256, normed=True)
        fft_hband2, hbins = np.histogram(fft_band2, bins=256, normed=True)

        hist_band1 = np.array([hband1]).ravel()
        series_band1 = pd.Series(hist_band1)
        hist_band2 = np.array([hband2]).ravel()
        series_band2 = pd.Series(hist_band2)

        fft_hist_band1 = np.array([fft_hband1]).ravel()
        fft_series_band1 = pd.Series(fft_hist_band1)
        fft_hist_band2 = np.array([fft_hband2]).ravel()
        fft_series_band2 = pd.Series(fft_hist_band2)

        diff = series_band1 - series_band2
        distance = np.sqrt(np.dot(diff, diff))
        row.append(distance)

        inter = histogram_intersection(hband1, hband2, hbins)
        row.append(inter)

        diff = fft_series_band1 - fft_series_band2
        distance = np.sqrt(np.dot(diff, diff))
        row.append(distance)

        inter = histogram_intersection(fft_hband1, fft_hband2, hbins)
        row.append(inter)

        manhattan = manhattan_distance(np.array([band1]).ravel(),np.array([band2]).ravel())
        row.append(manhattan)

        minkowski = minkowski_distance(np.array([band1]).ravel(),np.array([band2]).ravel(), 3)
        row.append(minkowski)

        cos_simil = cosine_similarity(np.array([band1]).ravel(),np.array([band2]).ravel())
        row.append(cos_simil)

        jaccard_simil = jaccard_similarity(np.array([band1]).ravel(),np.array([band2]).ravel())
        row.append(jaccard_simil)
        
        metrics.append(row)

    header = 'target', 'id', 'normal_dist','normal_intersec', 'fft_dist','fft_intersec', 'manhattan_dist', 'minkowski_dist', 'cosine_simil', 'jaccard_simil'
            
    with open("./test_metrics.csv", "wb") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(metrics)

Already done...


# Classic NN

## load metrics

In [6]:
train = pd.read_csv('./train_metrics.csv')
test = pd.read_csv('./test_metrics.csv')
num_train = train.shape[0]
train.head(5)

,target,id,normal_dist,normal_intersec,fft_dist,fft_intersec,manhattan_dist,minkowski_dist,cosine_simil,jaccard_simil
0,0,dfd5f913,76.651440,6.723106e+05,0.000053,1.0,665.890895,2.821,0.957,0.000582
1,0,e25388fd,34.406942,1.163699e+06,0.000002,1.0,384.697248,1.809,0.953,0.000314
2,1,58b2aaa0,53.654686,2.018744e+06,0.000010,1.0,721.904482,3.317,0.954,0.000423
3,0,4cfc3a18,61.308585,7.313449e+05,0.000043,1.0,448.786755,2.041,0.952,0.000488
4,0,271f93f4,58.743545,1.380521e+06,0.000031,1.0,784.238485,3.330,0.965,0.000435


## build datasets

In [7]:
dataset = train.values
X = dataset[:,2:10].astype(float)
y = dataset[:,0]

# normalize
X /= np.max(np.abs(X),axis=0)

## very simple model :-)

In [8]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(16, input_dim=8, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
seed = 7
np.random.seed(seed)

estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [ ]:
results = cross_val_score(estimator, X, y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 72.51% (1.43%)


##  larger model using gridsearch

### search for best # of epochs and batch size 

In [11]:
# larger model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(30, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, 
                        batch_size=10,
                        epochs=150,
                        verbose=2)

In [9]:
# grid search
epochs = [50, 100, 150]
batches = [5, 10, 20]
param_grid = dict(epochs=epochs, batch_size=batches)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X,y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Epoch 1/50
 - 2s - loss: 0.6893 - acc: 0.5538
Epoch 2/50
 - 1s - loss: 0.6809 - acc: 0.5566
Epoch 3/50
 - 1s - loss: 0.6603 - acc: 0.5837
Epoch 4/50
 - 1s - loss: 0.6258 - acc: 0.6146
Epoch 5/50
 - 1s - loss: 0.5974 - acc: 0.6511
Epoch 6/50
 - 1s - loss: 0.5782 - acc: 0.6660
Epoch 7/50
 - 1s - loss: 0.5678 - acc: 0.6810
Epoch 8/50
 - 1s - loss: 0.5636 - acc: 0.6810
Epoch 9/50
 - 1s - loss: 0.5596 - acc: 0.6810
Epoch 10/50
 - 1s - loss: 0.5583 - acc: 0.6745
Epoch 11/50
 - 1s - loss: 0.5512 - acc: 0.6885
Epoch 12/50
 - 1s - loss: 0.5529 - acc: 0.6763
Epoch 13/50
 - 1s - loss: 0.5477 - acc: 0.6857
Epoch 14/50
 - 1s - loss: 0.5496 - acc: 0.6848
Epoch 15/50
 - 1s - loss: 0.5454 - acc: 0.6904
Epoch 16/50
 - 1s - loss: 0.5434 - acc: 0.6819
Epoch 17/50
 - 1s - loss: 0.5391 - acc: 0.6932
Epoch 18/50
 - 1s - loss: 0.5437 - acc: 0.6857
Epoch 19/50
 - 1s - loss: 0.5402 - acc: 0.6894
Epoch 20/50
 - 1s - loss: 0.5370 - acc: 0.6988
Epoch 21/50
 - 1s - loss: 0.5373 - acc: 0.6932
Epoch 22/50
 - 1s - lo

Epoch 26/100
 - 1s - loss: 0.5446 - acc: 0.6904
Epoch 27/100
 - 1s - loss: 0.5393 - acc: 0.6913
Epoch 28/100
 - 1s - loss: 0.5346 - acc: 0.6941
Epoch 29/100
 - 1s - loss: 0.5357 - acc: 0.7025
Epoch 30/100
 - 1s - loss: 0.5305 - acc: 0.6922
Epoch 31/100
 - 1s - loss: 0.5387 - acc: 0.6960
Epoch 32/100
 - 1s - loss: 0.5337 - acc: 0.6997
Epoch 33/100
 - 1s - loss: 0.5312 - acc: 0.7007
Epoch 34/100
 - 1s - loss: 0.5359 - acc: 0.6922
Epoch 35/100
 - 1s - loss: 0.5271 - acc: 0.7109
Epoch 36/100
 - 1s - loss: 0.5294 - acc: 0.7016
Epoch 37/100
 - 1s - loss: 0.5266 - acc: 0.6960
Epoch 38/100
 - 1s - loss: 0.5285 - acc: 0.7025
Epoch 39/100
 - 1s - loss: 0.5263 - acc: 0.7025
Epoch 40/100
 - 1s - loss: 0.5266 - acc: 0.6997
Epoch 41/100
 - 1s - loss: 0.5263 - acc: 0.7100
Epoch 42/100
 - 1s - loss: 0.5273 - acc: 0.6988
Epoch 43/100
 - 1s - loss: 0.5221 - acc: 0.7091
Epoch 44/100
 - 1s - loss: 0.5202 - acc: 0.7081
Epoch 45/100
 - 1s - loss: 0.5192 - acc: 0.7072
Epoch 46/100
 - 1s - loss: 0.5189 - acc:

Epoch 97/100
 - 1s - loss: 0.4981 - acc: 0.7287
Epoch 98/100
 - 1s - loss: 0.4998 - acc: 0.7409
Epoch 99/100
 - 1s - loss: 0.5026 - acc: 0.7325
Epoch 100/100
 - 1s - loss: 0.4974 - acc: 0.7343
Epoch 1/100
 - 2s - loss: 0.6926 - acc: 0.5112
Epoch 2/100
 - 1s - loss: 0.6871 - acc: 0.5495
Epoch 3/100
 - 1s - loss: 0.6684 - acc: 0.6355
Epoch 4/100
 - 1s - loss: 0.6392 - acc: 0.6701
Epoch 5/100
 - 1s - loss: 0.6235 - acc: 0.6766
Epoch 6/100
 - 1s - loss: 0.5982 - acc: 0.7009
Epoch 7/100
 - 1s - loss: 0.5873 - acc: 0.6860
Epoch 8/100
 - 1s - loss: 0.5714 - acc: 0.7103
Epoch 9/100
 - 1s - loss: 0.5659 - acc: 0.7150
Epoch 10/100
 - 1s - loss: 0.5680 - acc: 0.7037
Epoch 11/100
 - 1s - loss: 0.5558 - acc: 0.7140
Epoch 12/100
 - 1s - loss: 0.5505 - acc: 0.7243
Epoch 13/100
 - 1s - loss: 0.5517 - acc: 0.7178
Epoch 14/100
 - 1s - loss: 0.5452 - acc: 0.7159
Epoch 15/100
 - 1s - loss: 0.5449 - acc: 0.7187
Epoch 16/100
 - 1s - loss: 0.5415 - acc: 0.7234
Epoch 17/100
 - 1s - loss: 0.5383 - acc: 0.7187


 - 1s - loss: 0.5155 - acc: 0.7156
Epoch 69/150
 - 1s - loss: 0.5094 - acc: 0.7175
Epoch 70/150
 - 1s - loss: 0.5178 - acc: 0.7138
Epoch 71/150
 - 1s - loss: 0.5060 - acc: 0.7240
Epoch 72/150
 - 1s - loss: 0.5097 - acc: 0.7175
Epoch 73/150
 - 1s - loss: 0.5126 - acc: 0.7231
Epoch 74/150
 - 1s - loss: 0.5073 - acc: 0.7306
Epoch 75/150
 - 1s - loss: 0.5077 - acc: 0.7297
Epoch 76/150
 - 1s - loss: 0.5127 - acc: 0.7240
Epoch 77/150
 - 1s - loss: 0.5047 - acc: 0.7287
Epoch 78/150
 - 1s - loss: 0.5039 - acc: 0.7222
Epoch 79/150
 - 1s - loss: 0.5058 - acc: 0.7156
Epoch 80/150
 - 1s - loss: 0.5104 - acc: 0.7250
Epoch 81/150
 - 1s - loss: 0.5094 - acc: 0.7362
Epoch 82/150
 - 1s - loss: 0.5059 - acc: 0.7166
Epoch 83/150
 - 1s - loss: 0.5072 - acc: 0.7409
Epoch 84/150
 - 1s - loss: 0.5060 - acc: 0.7240
Epoch 85/150
 - 1s - loss: 0.5024 - acc: 0.7222
Epoch 86/150
 - 1s - loss: 0.5044 - acc: 0.7268
Epoch 87/150
 - 1s - loss: 0.5074 - acc: 0.7231
Epoch 88/150
 - 1s - loss: 0.5048 - acc: 0.7362
Epoch

 - 1s - loss: 0.4906 - acc: 0.7343
Epoch 89/150
 - 1s - loss: 0.4931 - acc: 0.7418
Epoch 90/150
 - 1s - loss: 0.4995 - acc: 0.7250
Epoch 91/150
 - 1s - loss: 0.4938 - acc: 0.7437
Epoch 92/150
 - 1s - loss: 0.4956 - acc: 0.7343
Epoch 93/150
 - 1s - loss: 0.4946 - acc: 0.7353
Epoch 94/150
 - 1s - loss: 0.4904 - acc: 0.7297
Epoch 95/150
 - 1s - loss: 0.4890 - acc: 0.7381
Epoch 96/150
 - 1s - loss: 0.4911 - acc: 0.7343
Epoch 97/150
 - 1s - loss: 0.4903 - acc: 0.7409
Epoch 98/150
 - 1s - loss: 0.4874 - acc: 0.7418
Epoch 99/150
 - 1s - loss: 0.4959 - acc: 0.7315
Epoch 100/150
 - 1s - loss: 0.4919 - acc: 0.7456
Epoch 101/150
 - 1s - loss: 0.4868 - acc: 0.7399
Epoch 102/150
 - 1s - loss: 0.4866 - acc: 0.7428
Epoch 103/150
 - 1s - loss: 0.4879 - acc: 0.7446
Epoch 104/150
 - 1s - loss: 0.4893 - acc: 0.7399
Epoch 105/150
 - 1s - loss: 0.4913 - acc: 0.7428
Epoch 106/150
 - 1s - loss: 0.4916 - acc: 0.7278
Epoch 107/150
 - 1s - loss: 0.4964 - acc: 0.7381
Epoch 108/150
 - 1s - loss: 0.4860 - acc: 0.7

...

Epoch 9/100
 - 0s - loss: 0.6241 - acc: 0.6165
Epoch 10/100
 - 0s - loss: 0.6123 - acc: 0.6427
Epoch 11/100
 - 0s - loss: 0.6045 - acc: 0.6399
Epoch 12/100
 - 0s - loss: 0.5935 - acc: 0.6529
Epoch 13/100
 - 0s - loss: 0.5875 - acc: 0.6483
Epoch 14/100
 - 0s - loss: 0.5790 - acc: 0.6717
Epoch 15/100
 - 0s - loss: 0.5745 - acc: 0.6688
Epoch 16/100
 - 0s - loss: 0.5717 - acc: 0.6698
Epoch 17/100
 - 0s - loss: 0.5676 - acc: 0.6773
Epoch 18/100
 - 0s - loss: 0.5642 - acc: 0.6698
Epoch 19/100
 - 0s - loss: 0.5633 - acc: 0.6773
Epoch 20/100
 - 0s - loss: 0.5608 - acc: 0.6848
Epoch 21/100
 - 0s - loss: 0.5588 - acc: 0.6838
Epoch 22/100
 - 0s - loss: 0.5584 - acc: 0.6773
Epoch 23/100
 - 0s - loss: 0.5545 - acc: 0.6819
Epoch 24/100
 - 0s - loss: 0.5507 - acc: 0.6885
Epoch 25/100
 - 0s - loss: 0.5527 - acc: 0.6894
Epoch 26/100
 - 0s - loss: 0.5476 - acc: 0.6829
Epoch 27/100
 - 0s - loss: 0.5459 - acc: 0.6894
Epoch 28/100
 - 0s - loss: 0.5479 - acc: 0.6978
Epoch 29/100
 - 0s - loss: 0.5408 - acc: 

Epoch 80/100
 - 0s - loss: 0.4966 - acc: 0.7551
Epoch 81/100
 - 0s - loss: 0.5024 - acc: 0.7495
Epoch 82/100
 - 0s - loss: 0.5046 - acc: 0.7495
Epoch 83/100
 - 0s - loss: 0.5015 - acc: 0.7495
Epoch 84/100
 - 0s - loss: 0.4993 - acc: 0.7467
Epoch 85/100
 - 0s - loss: 0.4993 - acc: 0.7514
Epoch 86/100
 - 0s - loss: 0.5019 - acc: 0.7439
Epoch 87/100
 - 0s - loss: 0.4977 - acc: 0.7561
Epoch 88/100
 - 0s - loss: 0.5005 - acc: 0.7430
Epoch 89/100
 - 0s - loss: 0.5026 - acc: 0.7458
Epoch 90/100
 - 0s - loss: 0.5013 - acc: 0.7449
Epoch 91/100
 - 0s - loss: 0.4981 - acc: 0.7439
Epoch 92/100
 - 0s - loss: 0.4923 - acc: 0.7514
Epoch 93/100
 - 0s - loss: 0.4971 - acc: 0.7551
Epoch 94/100
 - 0s - loss: 0.4953 - acc: 0.7477
Epoch 95/100
 - 0s - loss: 0.4921 - acc: 0.7495
Epoch 96/100
 - 0s - loss: 0.4917 - acc: 0.7533
Epoch 97/100
 - 0s - loss: 0.4957 - acc: 0.7477
Epoch 98/100
 - 0s - loss: 0.4919 - acc: 0.7514
Epoch 99/100
 - 0s - loss: 0.4916 - acc: 0.7477
Epoch 100/100
 - 0s - loss: 0.4926 - acc

Epoch 150/150
 - 0s - loss: 0.4911 - acc: 0.7437
Epoch 1/150
 - 2s - loss: 0.6918 - acc: 0.5472
Epoch 2/150
 - 0s - loss: 0.6893 - acc: 0.5463
Epoch 3/150
 - 0s - loss: 0.6866 - acc: 0.5463
Epoch 4/150
 - 0s - loss: 0.6811 - acc: 0.5463
Epoch 5/150
 - 0s - loss: 0.6687 - acc: 0.5641
Epoch 6/150
 - 0s - loss: 0.6513 - acc: 0.6006
Epoch 7/150
 - 0s - loss: 0.6306 - acc: 0.6146
Epoch 8/150
 - 0s - loss: 0.6157 - acc: 0.6361
Epoch 9/150
 - 0s - loss: 0.6080 - acc: 0.6277
Epoch 10/150
 - 0s - loss: 0.5960 - acc: 0.6567
Epoch 11/150
 - 0s - loss: 0.5886 - acc: 0.6548
Epoch 12/150
 - 0s - loss: 0.5812 - acc: 0.6623
Epoch 13/150
 - 0s - loss: 0.5747 - acc: 0.6717
Epoch 14/150
 - 0s - loss: 0.5694 - acc: 0.6698
Epoch 15/150
 - 0s - loss: 0.5633 - acc: 0.6735
Epoch 16/150
 - 0s - loss: 0.5608 - acc: 0.6782
Epoch 17/150
 - 0s - loss: 0.5572 - acc: 0.6810
Epoch 18/150
 - 0s - loss: 0.5531 - acc: 0.6904
Epoch 19/150
 - 0s - loss: 0.5500 - acc: 0.6941
Epoch 20/150
 - 0s - loss: 0.5480 - acc: 0.6969


Epoch 20/150
 - 0s - loss: 0.5420 - acc: 0.7252
Epoch 21/150
 - 0s - loss: 0.5424 - acc: 0.7196
Epoch 22/150
 - 0s - loss: 0.5383 - acc: 0.7206
Epoch 23/150
 - 0s - loss: 0.5473 - acc: 0.7159
Epoch 24/150
 - 0s - loss: 0.5396 - acc: 0.7215
Epoch 25/150
 - 0s - loss: 0.5341 - acc: 0.7187
Epoch 26/150
 - 0s - loss: 0.5350 - acc: 0.7224
Epoch 27/150
 - 0s - loss: 0.5368 - acc: 0.7168
Epoch 28/150
 - 0s - loss: 0.5332 - acc: 0.7308
Epoch 29/150
 - 0s - loss: 0.5278 - acc: 0.7308
Epoch 30/150
 - 0s - loss: 0.5278 - acc: 0.7364
Epoch 31/150
 - 0s - loss: 0.5284 - acc: 0.7299
Epoch 32/150
 - 0s - loss: 0.5373 - acc: 0.7271
Epoch 33/150
 - 0s - loss: 0.5271 - acc: 0.7290
Epoch 34/150
 - 0s - loss: 0.5229 - acc: 0.7411
Epoch 35/150
 - 0s - loss: 0.5216 - acc: 0.7430
Epoch 36/150
 - 0s - loss: 0.5233 - acc: 0.7336
Epoch 37/150
 - 0s - loss: 0.5232 - acc: 0.7439
Epoch 38/150
 - 0s - loss: 0.5193 - acc: 0.7393
Epoch 39/150
 - 0s - loss: 0.5200 - acc: 0.7374
Epoch 40/150
 - 0s - loss: 0.5194 - acc:

Epoch 40/150
 - 1s - loss: 0.5078 - acc: 0.7232
Epoch 41/150
 - 1s - loss: 0.5081 - acc: 0.7263
Epoch 42/150
 - 1s - loss: 0.5124 - acc: 0.7170
Epoch 43/150
 - 1s - loss: 0.5067 - acc: 0.7244
Epoch 44/150
 - 1s - loss: 0.5069 - acc: 0.7219
Epoch 45/150
 - 1s - loss: 0.5072 - acc: 0.7257
Epoch 46/150
 - 1s - loss: 0.5069 - acc: 0.7251
Epoch 47/150
 - 1s - loss: 0.5093 - acc: 0.7232
Epoch 48/150
 - 1s - loss: 0.5108 - acc: 0.7338
Epoch 49/150
 - 1s - loss: 0.5043 - acc: 0.7357
Epoch 50/150
 - 1s - loss: 0.5081 - acc: 0.7282
Epoch 51/150
 - 1s - loss: 0.5068 - acc: 0.7282
Epoch 52/150
 - 1s - loss: 0.5072 - acc: 0.7282
Epoch 53/150
 - 1s - loss: 0.5063 - acc: 0.7288
Epoch 54/150
 - 1s - loss: 0.5027 - acc: 0.7213
Epoch 55/150
 - 1s - loss: 0.5017 - acc: 0.7313
Epoch 56/150
 - 1s - loss: 0.5069 - acc: 0.7282
Epoch 57/150
 - 1s - loss: 0.5050 - acc: 0.7257
Epoch 58/150
 - 1s - loss: 0.5058 - acc: 0.7288
Epoch 59/150
 - 1s - loss: 0.5010 - acc: 0.7350
Epoch 60/150
 - 1s - loss: 0.4999 - acc:

### search for best optimizer

In [12]:
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, y)
                      
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

ValueError: optimizer is not a legal parameter